In [1]:
import random

### Importing function for generating chromosome

In [2]:
from CGen import *

In [3]:
%%time
L,B,H=5,5,5

P1=[]

for _ in range(10):
    CH=generate_chromosome(L,B,H,random.random())
    P1.append(CH)
    get_stats(CH)
    print('-'*125)

(R,G,B,A) = (167,140,246,0.9921935843504812)
Roughness Factor = 0.4094527118315827
Number of vertices = 44
Number of faces = 454
-----------------------------------------------------------------------------------------------------------------------------
(R,G,B,A) = (198,41,99,0.04645623820340117)
Roughness Factor = 0.8680561957232166
Number of vertices = 52
Number of faces = 670
-----------------------------------------------------------------------------------------------------------------------------
(R,G,B,A) = (179,126,86,0.9326265314993207)
Roughness Factor = 0.30538731373300665
Number of vertices = 47
Number of faces = 532
-----------------------------------------------------------------------------------------------------------------------------
(R,G,B,A) = (134,8,220,0.49791092721965935)
Roughness Factor = 0.8618836433031104
Number of vertices = 36
Number of faces = 315
------------------------------------------------------------------------------------------------------------

## Mutation

In [81]:
def mutate(C):
    for i in range(len(C)):
        if random.random()>0.5:
            

IndentationError: expected an indented block (Temp/ipykernel_28412/3924105588.py, line 2)